In [2]:
xb = XgboostFeature()

Xgboost Feature start, new_feature number: 30


In [3]:
sys.path.append('/Users/lin/Enterprise/missingno-master/missingno')
import missingno as msno

In [4]:
np.set_printoptions(suppress=True)

In [5]:
startID = ['f20','f24','f28','f32','f36','f48','f52','f54',
           'f64','f72','f76','f102','f107','f111','f155','f161','f166',
          'f211','f254','f278']

In [6]:
catid = ['f1','f2','f3','f4']

In [7]:
userid = ['f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']

In [8]:
part = []
for i in range(166,278):
        part.append("f" + str(i))


In [9]:
print("Split remain columns into 3 groups!")
cat_list = ['f1','f2','f3','f4','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15','f16','f17','f18','f19']#分类性
reg_list = ['f182','f183','f184','f185','f204','f205','f206','f207','f259','f260','f261','f270','f271']#离散性
ind_list = ['f161','f162','f163','f164','f165']#连续性
for i in range(211,254):
        ind_list.append("f" + str(i))

Split remain columns into 3 groups!


In [10]:
def createCol():
    col = ['label','date']
    for i in range(1,298):
        col.append("f" + str(i))
    
    return col

In [11]:
def creatDtype():
    dtype = {
        'id':'object',
        'label':'int8',
        'date':'int64',
        'f1' : 'uint8',
        'f2' : 'uint8',
        'f3' : 'uint8',
        'f4' : 'uint8',
        'f5' : 'float32'
    }
    for i in range(20,298):
        dtype["f" + str(i)] = 'float32'
    for i in range(6,20):
        dtype["f" + str(i)] = 'uint8'
    
    return dtype

In [12]:
col_list = []
for i in range(161,278):
    col_list.append("f"+str(i))
    
    

In [13]:
print("loading data!")
train = pd.read_csv('atec_anti_fraud_train.csv', header = 0,dtype=creatDtype(),usecols=createCol())
test = pd.read_csv('atec_anti_fraud_test_b.csv',header = 0,dtype=creatDtype())
#test1 = pd.read_csv('atec_anti_fraud_test_b.csv',header = 0,dtype=creatDtype())
print(train.shape)

loading data!
(994731, 299)


In [14]:
# train = train.sort_values(by=['date'])
# train = train[train.date < 20171016]

In [15]:
# train = train.reset_index(drop = True)

In [16]:
# print("Delete -1 in label!")
# train.drop(np.where(train.label == -1)[0],inplace=True)
# train = train.reset_index(drop = True)
# print(train.shape)

In [17]:
train = train.replace(-1,1)

In [18]:
# test['label'] = 2

In [19]:
print("Take off label!")
label = train['label']
train = train.drop(['label'],axis=1)
print(train.shape)
gc.collect()

Take off label!
(994731, 298)


7

In [20]:
print("Concat train and test!")
len_train = len(train)
test = test.drop(['id'],axis=1)
train = train.append(test)
train = train.reset_index(drop=True)
print(train.shape)
del test
gc.collect()

Concat train and test!
(1495269, 298)


7

In [21]:
print("Counting f82!")
gp = train[userid + ['f82']].groupby(by=userid)['f82'].count().reset_index().rename(columns={'f82': 'money_count'})
print("Merging!")
train = train.merge(gp, on=userid, how='left')
train['money_count'] = train['money_count'].astype('uint16')
print(train.shape)
del gp
gc.collect()

Counting f82!
Merging!
(1495269, 299)


206

In [22]:
print("Count f1-f4!")
train['f300'] = ''
gp = train[['f1','f2','f3','f4','f300']].groupby(by=['f1','f2','f3','f4'])['f300'].count().reset_index().rename(columns={'f300': 'count1_4'})
print("Merging!")
train = train.merge(gp, on=['f1','f2','f3','f4'], how='left')
train['count1_4'] = train['count1_4'].astype('uint16')
train = train.drop(['f300'],axis=1)
print(train.shape)
del gp
gc.collect()

Count f1-f4!
Merging!
(1495269, 300)


136

In [23]:
print("Count f6-f19!")
train['f300'] = ''
gp = train[userid + ['f300']].groupby(userid)['f300'].count().reset_index().rename(columns={'f300': 'count6_19'})
print("Merging!")
train = train.merge(gp, on=userid, how='left')
train['count6_19'] = train['count6_19'].astype('uint16')
train = train.drop(['f300'],axis=1)
print(train.shape)
del gp
gc.collect()

Count f6-f19!
Merging!
(1495269, 301)


240

In [24]:
# print("Count f1-f19!")
# col = userid + ['f1','f2','f3','f4']
# train['f300'] = ''
# gp = train[col + ['f300']].groupby(col)['f300'].count().reset_index().rename(columns={'f300': 'count1_19'})
# print("Merging!")
# train = train.merge(gp, on=col, how='left')
# train['count1_19'] = train['count1_19'].astype('uint16')
# train = train.drop(['f300'],axis=1)
# print(train.shape)
# del gp
# gc.collect()

In [25]:
# print("Counting 6-19 for each part!")
# for i in startID:
#     gp = train[userid + [i]].groupby(by=userid)[i].count().reset_index().rename(columns={i: 'count6_19'+ i})
#     train = train.merge(gp, on=userid, how='left')
#     train['count6_19'+ i] = train['count6_19'+ i].astype('uint16')
#     del gp
#     gc.collect()
# print(train.shape)

In [26]:
# print("Counting 1-4 for each part!")
# for i in startID:
#     gp = train[catid + [i]].groupby(by=catid)[i].count().reset_index().rename(columns={i: 'count1_4'+ i})
#     train = train.merge(gp, on=catid, how='left')
#     train['count1_4'+ i] = train['count1_4'+ i].astype('uint16')
#     del gp
#     gc.collect()
# print(train.shape)

In [27]:
# print("Counting 1-19 for each part!")
# allid = userid+catid
# for i in startID:
#     gp = train[allid + [i]].groupby(by=allid)[i].count().reset_index().rename(columns={i: 'count1_19'+ i})
#     train = train.merge(gp, on=allid, how='left')
#     train['count1_19'+ i] = train['count1_19'+ i].astype('uint16')
#     del gp
#     gc.collect()
# print(train.shape)

In [28]:
print("Calculate numbers of Nan in each line")
train['count_nan'] = train.T.isnull().sum()
train['count_nan'] = train['count_nan'].astype('uint16')
print(train.shape)
gc.collect()

Calculate numbers of Nan in each line
(1495269, 302)


7

In [29]:
print("Calculate percent of Nan in each line!")
train['nan_percent'] = train['count_nan'] / 300.0
train['nan_percent'] = train['nan_percent'].astype('float32')
print(train.shape)
gc.collect()

Calculate percent of Nan in each line!
(1495269, 303)


7

In [30]:
print("Delete col with too much NaN!")
delcol = []
for i in train.columns:
    if(float(train[i].isnull().sum()) / len(train) > 0.3):
        delcol.append(i)
print("Delcol number is ", len(delcol))
train = train.drop(delcol,axis=1)
del delcol
print(train.shape)
gc.collect()

Delete col with too much NaN!
('Delcol number is ', 127)
(1495269, 176)


7

In [31]:
# print("Creating occurance features!")
# train = train.fillna(-1)
# for i in col_list:
#     print("deal with",i)
#     f_ratio = pd.DataFrame(train[i].value_counts())
#     #f_ratio[i + '_rcount'] = f_ratio[i]
#     f_ratio[i + '_ratio'] = f_ratio[i]/len(train)
#     f_ratio[i + '_ratio'] = f_ratio[i + '_ratio'].astype('float32')
#     f_ratio[i] = f_ratio.index
#     f_ratio = f_ratio.reset_index(drop = True)
#     train = train.merge(f_ratio,on = i,how = 'left')
# gc.collect()

In [32]:
# data_corr = train.corr()

# # Set the threshold to select only highly correlated attributes
# threshold = 0.9

# # List of pairs along with correlation above threshold
# corr_list = []
# size = len(train.columns)
# #Search for the highly correlated pairs
# for i in range(0,size): #for 'size' features
#     for j in range(i+1,size): #avoid repetition
#         if (data_corr.iloc[i,j] >= threshold and data_corr.iloc[i,j] < 1) or (data_corr.iloc[i,j] < 0 and data_corr.iloc[i,j] <= -threshold):
#             corr_list.append([data_corr.iloc[i,j],i,j]) #store correlation and columns index

# #Sort to show higher ones first            
# s_corr_list = sorted(corr_list,key=lambda x: -abs(x[0]))

# #Print correlations and column names
# # for v,i,j in s_corr_list:
# #     print ("%s and %s = %.2f" % (train.columns[i],train.columns[j],v))

In [33]:
# print("Filling Nan!")
# train = train.replace(-1,np.nan)
# train.isnull().any().any()

In [34]:
# print("Filling Nan by LGB!")
# predict_features = []
# for i in train.columns:
#     if(train[i].isnull().sum() != 0):
#         #if(float(train[i].isnull().sum()) / len(train) < 0.2):
#         predict_features.append(i)
# train = set_missing(train,predict_features)

# train.isnull().any().any()

In [35]:
train = train.fillna(-1)
train.isnull().any().any()

False

In [36]:
# print("Getting last trading time delta!")
# train = train.sort_values(['date'])
# train = train.reset_index(drop=True)
# train['year'] = train['date'].values / 10000
# train['month'] = train['date'].values /100 % 100
# train['day'] = train['date'].values % 100
# train['convertDays'] = train['year'].values * 365 + (train['month'].values - 1)* 30 + train['day'].values
# train['time_delta'] = train.sort_values(['date']).groupby(by=userid)[['convertDays']].diff()
# train = train.drop(['month','day','year','convertDays'],axis=1)
# gc.collect()

In [37]:
# print("Getting trade times!")
# gp = train[userid + ['date']].groupby(by=userid)['date'].count().reset_index().rename(columns={'date': 'trade_count'})
# print("Merging!")
# train = train.merge(gp, on=userid, how='left')
# train['trade_count'] = train['trade_count'].astype('uint16')
# del gp
# gc.collect()

In [38]:
# print("Getting trade unique times!")
# gp = train[userid + ['date']].groupby(by=userid)['date'].nunique().reset_index().rename(columns={'date': 'trade_unique'})
# print("Merging!")
# train = train.merge(gp, on=userid, how='left')
# train['trade_unique'] = train['trade_unique'].astype('uint16')
# del gp
# gc.collect()

In [39]:
# print("Getting trade cumcount times!")
# train = train.sort_values(['date'])

# train = train.reset_index(drop = True)
# train['trade_cumcount'] = train[userid + ['date']].groupby(by=userid)['date'].cumcount() + 1
# train['trade_cumcount'] = train['trade_cumcount'].astype('uint16')
# gc.collect()

In [40]:
# print("Getting first trade time")
# train_ = train[userid + ['date','trade_cumcount']]
# train_ = train_.sort_values(by=['trade_cumcount']).reset_index(drop = True)
# train_ = train_[:(np.where(train_['trade_cumcount'] ==2)[0][0])]
# train_['first_trade_time'] = train_['date'] * train_['trade_cumcount']
# train_ = train_.drop(['date','trade_cumcount'],axis=1) 
# train = train.merge(train_,on=userid, how='left')
# del train_
# train['trade_cumcount'] = train['trade_cumcount'] - 1
# gc.collect()

In [41]:
# print("Get important features!")
# group1 = ['f204','f205','f206','f207']
# group2 = ['f208','f209','f210']
# group3 = ['f215','f216','f217','f218']
# group4 = ['f234','f235','f236','f237','f238']
# group5 = ['f243','f244','f245','f246','f247','f248','f253']
# group6 = ['f259','f260','f261','f262','f263','f266']
# groups = [group1,group2,group3,group4,group5,group6]

In [42]:
# print("Doing interaction between important features!")
# for i in groups:
#     print("Dealing with",i)
#     for e, (x, y) in enumerate(combinations(i, 2)):
#         interaction_features(train,x,y,e)
#     print(train.shape)
# gc.collect()

In [43]:
# print("Getting proj features!")
# for i in group1:
#     print("Dealing with",i,"and f7")
#     df = proj_num_on_cat(train,i,'f7')
#     for i in df.columns:
#         df[i] = df[i].astype('uint32')
#     train = pd.concat([train,df],axis = 1)
#     print("train shape",train.shape)
# del df
# gc.collect()

In [44]:
# print("One_Hot_Encoder!")
# ohe = []
# #need to change to 1-19
# col = userid + ['f1','f2','f3','f4']
# for column in col:
#     temp = pd.get_dummies(pd.Series(train[column]))
#     for i in temp.columns:
#         temp.rename(columns = {i : column + '_' + str(i)}, inplace = True)
#         ohe.append(column + '_' + str(i))
#     train = pd.concat([train,temp],axis=1)
#     train = train.drop([column],axis=1)

# print(len(ohe))
# gc.collect()

In [45]:
# print("Counting cat features!")
# col = userid + ['f1','f2','f3','f4']
# cat_count_df = cat_count(train, col)
# for i in cat_count_df.columns:
#     cat_count_df[i] = cat_count_df[i].astype('uint32')
# train = pd.concat([train,cat_count_df],axis=1)
# del cat_count_df
# gc.collect()

In [46]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1495269 entries, 0 to 1495268
Columns: 176 entries, date to nan_percent
dtypes: float32(153), int64(1), uint16(4), uint8(18)
memory usage: 932.6 MB


In [47]:
# print("Starting subsample!")
# test = train[len_train:]
# train = train[:len_train]
# print("train size",len(train))
# print("test size",len(test))
# # target = pd.DataFrame(target)
# # train = pd.concat([train,target],axis=1)
# print(train.shape,test.shape)
# print("Split into 0 and 1!")
# train0 = train[train['label'] == 0]
# train1 = train[train['label'] == 1]
# train0_X,val0_X, train0_y, val0_y = train_test_split(train0,train0['label'].values,test_size = int(len(train) *5),random_state = 0) 
# df = pd.DataFrame(val0_X,columns=train.columns)
# print("DF with 0 label shape", df.shape)
# train_ = df.append(train1)
# train_ = train.reset_index(drop=True)
# print("Preparing data for training!")
# target = train_['label'].values
# train_ = train_.drop(['label'],axis=1)
# X = train_.values
# y = target
# print("X",X.shape,"Y",y.shape)
# sub = pd.read_csv('atec_anti_fraud_test_a.csv', usecols=['id'])
# sub['score']=0
# gc.collect()

In [48]:
# print("Dealing with -1 label!")
# train = train.drop(['date'],axis = 1)
# test = train[len_train:]
# train = train[:len_train]
# test = test.drop(['label'],axis = 1)
# df = train[train['label'] == -1]

# print("Delete -1 in train!")
# train.drop(np.where(train.label == -1)[0],inplace=True)
# train = train.reset_index(drop = True)
# df = df.reset_index(drop = True)

# print("Drop label!")
# df = df.drop(['label'],axis=1)
# label = train['label'].values
# train = train.drop(['label'],axis=1)

# print("train shape",train.shape)
# print("test shape",test.shape)
# print("df shape",df.shape)

# train_X,val_X, train_y, val_y = train_test_split(train,label,test_size = 0.25,random_state = 0) 

# print("Preparing model!")
# clf = lgb.LGBMClassifier(
#         boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,
#         max_depth=-1, n_estimators=3000, objective='binary',
#         subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
#         learning_rate=0.05, min_child_weight=50, random_state=2018, n_jobs=100
#     )
# clf.fit(train_X.values,train_y,eval_set=[(val_X.values, val_y)],eval_metric='auc',early_stopping_rounds=30,verbose=100)
# result = clf.predict(df.values,num_iteration=clf.best_iteration_)

# print("Preparing data for training!")
# df['label'] = result
# train['label'] = label
# train = train.append(df)
# train = train.reset_index(drop=True)
# label = train['label']
# train = train.drop(['label'],axis=1)

# X = train.values
# y= label.values
# print("X",X.shape,"Y",y.shape)
# print("Train size",train.shape)
# print("Test size",test.shape)

# print("Preparing sub!")
# sub = pd.read_csv('atec_anti_fraud_test_a.csv', usecols=['id'])
# sub['score']=0

# del train
# gc.collect()
# print("All done!")

In [49]:
print("Preparing data for training!")
train = train.drop(['date'],axis = 1)
#train = train[:len_test]
print train.shape
test = train[len_train:]
train = train[:len_train]
y = label.values
#train = train.drop(['label'],axis=1)
X = train.values
print("X",X.shape,"Y",y.shape)
print("Train size",train.shape)
print("Test size",test.shape)
sub = pd.read_csv('atec_anti_fraud_test_a.csv', usecols=['id'])
sub['score']=0
del train

gc.collect()

Preparing data for training!
(1495269, 175)
('X', (994731, 175), 'Y', (994731,))
('Train size', (994731, 175))
('Test size', (500538, 175))


28

In [50]:
# print("Creating XGB features!")
# print("Current X",X.shape,"test",test.shape)
# [X,y,test] = xb.fit_model(X,y,test.values)
# print("After Current X",X.shape,"test",test.shape)
# gc.collect()

In [51]:
# print("XGB!")
# xgbparams = {'eta': 0.1, 
#           'tree_method': "auto", 
#           'max_depth': 8, 
#           'subsample': 0.7, 
#           'colsample_bytree': 0.7, 
#           'colsample_bylevel':0.1,
#           'min_child_weight':16,
#           #'alpha':4,
#           'objective': 'binary:logistic', 
#           'scale_pos_weight':99,
#           'eval_metric': 'auc', 
#           'random_state': 99,
#           'silent': True}

In [52]:
# print("XGB!")
# xgbparams = {'eta': 0.1, 
#           'tree_method': "auto", 
#           'max_depth': 4, 
#           'subsample': 0.7, 
#           'colsample_bytree': 0.8, 
#           'colsample_bylevel':0.8,
#           'min_child_weight':10,
#           'gamma':0.6,   
#           #'reg_alpha':1e-05,
#           #'reg_lambda':1,
#           'objective': 'binary:logistic', 
#           'scale_pos_weight':99,
#           'eval_metric': 'auc', 
#           'random_state': 99,
#           'silent': True}

In [53]:
# train_X,val_X, train_y, val_y = train_test_split(train,target,test_size = 0.1,random_state = 0) 
# train_y = train_y.astype('uint8')
# val_y = val_y.astype('uint8')
# print('The size of the test set is ', len(test))
# print('The size of the validation set is ', len(val_X))
# print('The size of the train set is ', len(train_X))
# dtrain = xgb.DMatrix(train_X, train_y)
# del train_X, train_y
# dvalid = xgb.DMatrix(val_X, val_y)
# del val_X, val_y
# watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
# model = xgb.train(xgbparams, dtrain, 1000, watchlist, maximize=True, early_stopping_rounds=20, verbose_eval=10)
# del dtrain
# del dvalid
# gc.collect()

In [54]:
# def ceate_feature_map(features):  
#     outfile = open('xgb.fmap', 'w')  
#     i = 0  
#     for feat in features:  
#         outfile.write('{0}\t{1}\tq\n'.format(i, feat))  
#         i = i + 1  
#     outfile.close() 
    
# print("saveing feat_importance!")
# features = [x for x in train.columns if x not in ['id','loss']]  
# ceate_feature_map(features)  
  
# importance = xgb_model.get_fscore(fmap='xgb.fmap')  
# importance = sorted(importance.items(), key=operator.itemgetter(1))  
  
# df = pd.DataFrame(importance, columns=['feature', 'fscore'])  
# df['fscore1'] = df['fscore'] / df['fscore'].sum()  
# df.to_csv("feat_importance.csv", index=False)  

# plt.figure()  
# df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(50, 50))  
# plt.title('XGBoost Feature Importance1')  
# plt.xlabel('relative importance')  
# plt.savefig("feat_importance.jpg",dpi = 400)

In [55]:
# X_train = X[0:length]
# X_valid = X[length:]
# y_train = y[0:length]
# y_valid = y[length:]
d_train = xgb.DMatrix(X_train, y_train) 
d_valid = xgb.DMatrix(X_valid, y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
xgb_model = xgb.train(xgbparams, d_train, 5000, watchlist, early_stopping_rounds=100,maximize=True, verbose_eval=100)
# sub['score'] = xgb_model.predict(xgb.DMatrix(test.values), ntree_limit=xgb_model.best_ntree_limit +150)

# sub.to_csv('sub_sinxgb_xgbfit_nt_params.csv',index=False)
# print("Is this Done? No, he fucked up!")

In [56]:
xgbparams = {'eta': 0.1, 
          'tree_method': "auto", 
          'max_depth': 4, 
          'subsample': 0.7, 
          'colsample_bytree': 0.1, 
          'colsample_bylevel':0.1,
          'min_child_weight':10,
          'gamma':0.6,   
          #'reg_alpha':1e-05,
          #'reg_lambda':1,
          'objective': 'binary:logistic', 
          'scale_pos_weight':1,
          'eval_metric': 'auc', 
          'random_state': 99,
          'silent': True}

In [57]:
print("K-fold training!")
sub = pd.read_csv('atec_anti_fraud_test_b.csv', usecols=['id'])
sub['score']=0
#del train
#stacking = train.f1
#print("stacking length",len(stacking))
#result = np.zeros(len(test))
#print("stacking result length",len(result))
bst_xgb = []
nrounds=5000  
kfold = 5  
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print("train index",train_index)
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(xgbparams, d_train, nrounds, watchlist, early_stopping_rounds=70,maximize=True, verbose_eval=100)
    sub['score'] += xgb_model.predict(xgb.DMatrix(test.values), ntree_limit= xgb_model.best_ntree_limit+160) / (2 * kfold)
    #result += xgb_model.predict(xgb.DMatrix(test), ntree_limit=xgb_model.best_ntree_limit +150) / (kfold)
    #bst_xgb.append(xgb_model.best_ntree_limit)
    #stacking[test_index] = xgb_model.predict(d_valid, ntree_limit=xgb_model.best_ntree_limit +150)

    
gc.collect()

K-fold training!
('train index', array([198937, 198938, 198939, ..., 994728, 994729, 994730]))
 xgb kfold: 1  of  5 : 
[0]	train-auc:0.664066	valid-auc:0.657432
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 70 rounds.
[100]	train-auc:0.955802	valid-auc:0.956845
[200]	train-auc:0.965596	valid-auc:0.966013
[300]	train-auc:0.97111	valid-auc:0.970631
[400]	train-auc:0.973592	valid-auc:0.972529
[500]	train-auc:0.975135	valid-auc:0.973748
[600]	train-auc:0.976169	valid-auc:0.97451
[700]	train-auc:0.977058	valid-auc:0.975087
[800]	train-auc:0.97773	valid-auc:0.975588
[900]	train-auc:0.97834	valid-auc:0.975963
[1000]	train-auc:0.978714	valid-auc:0.976164
[1100]	train-auc:0.979148	valid-auc:0.9764
[1200]	train-auc:0.97953	valid-auc:0.976612
[1300]	train-auc:0.979889	valid-auc:0.97679
[1400]	train-auc:0.980211	valid-auc:0.977009
[1500]	train-auc:0.980494	valid-auc:0.977134
[1600]	train-auc:0.980774	valid-auc:0.

[900]	train-auc:0.977695	valid-auc:0.975353
[1000]	train-auc:0.978545	valid-auc:0.976045
[1100]	train-auc:0.979189	valid-auc:0.97656
[1200]	train-auc:0.97957	valid-auc:0.976726
[1300]	train-auc:0.979897	valid-auc:0.976955
[1400]	train-auc:0.980314	valid-auc:0.97726
[1500]	train-auc:0.980631	valid-auc:0.977426
[1600]	train-auc:0.980923	valid-auc:0.977539
[1700]	train-auc:0.9812	valid-auc:0.977728
[1800]	train-auc:0.98149	valid-auc:0.977844
[1900]	train-auc:0.981733	valid-auc:0.977984
[2000]	train-auc:0.981983	valid-auc:0.978162
[2100]	train-auc:0.982216	valid-auc:0.978226
[2200]	train-auc:0.982413	valid-auc:0.978326
[2300]	train-auc:0.982653	valid-auc:0.978498
[2400]	train-auc:0.982858	valid-auc:0.978587
[2500]	train-auc:0.983062	valid-auc:0.978698
Stopping. Best iteration:
[2506]	train-auc:0.983067	valid-auc:0.978709



28

In [58]:
for i in range(0,5):
    print i
    sub['score'] += xgb_model.predict(xgb.DMatrix(test.values), ntree_limit=xgb_model.best_ntree_limit+160) / (2*kfold)

0
1
2
3
4


In [59]:
sub.to_csv('sub_xgb_40b_1.csv',index=False)
print("Is this Done? No, he fucked up!")

Is this Done? No, he fucked up!


In [60]:
# print("Stacking!")
# test = convertToOne(test,result)
# X = convertToOne(X,stacking.values)
# print("After Current X",X.shape,"test",test.shape)
# print("Training!")
# xgb_model = xgb.train(xgbparams,xgb.DMatrix(train.values, y), int(np.sum(bst_xgb) / 5 *1.1), verbose_eval=50)
# print("Predicting!")
# sub = pd.read_csv('atec_anti_fraud_test_a.csv', usecols=['id'])
# sub['score']=0
# sub['score'] = xgb_model.predict(xgb.DMatrix(test.values))
# sub.to_csv('sub_xgb_me_nt_stacking.csv',index=False)
# print("Is this Done? No, he fucked up!")

In [61]:
# lgbparams = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.1,
#     'num_leaves': 31,  
#     'max_depth': -1,  
#     'min_child_samples': 100,  
#     'max_bin': 100,  
#     'subsample': 0.7,  
#     'subsample_freq': 1,  
#     'colsample_bytree': 0.7,  
#     'min_child_weight': 10,  
#     #'subsample_for_bin': 200000,  
#     'min_split_gain': 0,  
#     'reg_alpha': 0,  
#     'reg_lambda': 0,  
#    # 'nthread': 8,
#     'verbose': 0,
#     'scale_pos_weight':100 
#     }


In [62]:
# sub['score']=0
# print("K-fold lgb training!")
# kfold = 5   
# nrounds = 2000
# skf = StratifiedKFold(n_splits=kfold, random_state=0)
# for i, (train_index, test_index) in enumerate(skf.split(X, y)):
#     print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
#     X_train, X_valid = X[train_index], X[test_index]
#     y_train, y_valid = y[train_index], y[test_index]
#     lgb_model = lgb.train(lgbparams, lgb.Dataset(X_train, label=y_train), nrounds, 
#                   lgb.Dataset(X_valid, label=y_valid), verbose_eval=100, early_stopping_rounds=30)
#     sub['score'] += lgb_model.predict(test.values,num_iteration=lgb_model.best_iteration) / (kfold)
# gc.collect()

In [63]:
# sub.to_csv('sub_lgb_me_-1_notime.csv',index=False)
# print("Is this Done? No, he fucked up!")

In [64]:
## print("Tuning!")
# # tuning
# ITERATIONS = 1000 # 1000
# # Classifier
# bayes_cv_tuner = BayesSearchCV(
#     estimator = xgb.XGBClassifier(
#         n_jobs = 1,
#         objective = 'binary:logistic',
#         eval_metric = 'auc',
#         silent=1,
#         tree_method='approx'
#     ),
#     search_spaces = {
#         'learning_rate': (0.01, 1.0, 'log-uniform'),
#         'min_child_weight': (0, 10),
#         'max_depth': (0, 50),
#         'max_delta_step': (0, 20),
#         'subsample': (0.01, 1.0, 'uniform'),
#         'colsample_bytree': (0.01, 1.0, 'uniform'),
#         'colsample_bylevel': (0.01, 1.0, 'uniform'),
#         'reg_lambda': (1e-9, 1000, 'log-uniform'),
#         'reg_alpha': (1e-9, 1.0, 'log-uniform'),
#         'gamma': (1e-9, 0.5, 'log-uniform'),
#         'min_child_weight': (0, 5),
#         'n_estimators': (50, 100),
#         'scale_pos_weight': (1e-6, 500, 'log-uniform')
#     },    
#     scoring = 'roc_auc',
#     cv = StratifiedKFold(
#         n_splits=3,
#         shuffle=True,
#         random_state=42
#     ),
#     n_jobs = 3,
#     n_iter = ITERATIONS,   
#     verbose = 0,
#     refit = True,
#     random_state = 42
# )
# def status_print(optim_result):
#     """Status callback durring bayesian hyperparameter search"""
    
#     # Get all the models tested so far in DataFrame format
#     all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
#     # Get current parameters and the best parameters    
#     best_params = pd.Series(bayes_cv_tuner.best_params_)
#     print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
#         len(all_models),
#         np.round(bayes_cv_tuner.best_score_, 4),
#         bayes_cv_tuner.best_params_
#     ))
    
#     # Save all model results
#     clf_name = bayes_cv_tuner.estimator.__class__.__name__
#     all_models.to_csv(clf_name+"_cv_results.csv")

# result = bayes_cv_tuner.fit(X, y, callback=status_print)


    